In [1]:
#!pip install tpot autogluon flaml

Random State for reproduceability

In [2]:
dagstuhl_seed=23372

# Data

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('../data/preprocessed.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_train = pd.read_csv('../data/preprocessed-train.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]
df_train = df_train.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df_test = pd.read_csv('../data/preprocessed-test.csv')
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]
df_test = df_test.replace(['inactive', 'active'], [0,1]) # swap -1 (inactive) and 0 (unknown)

df.tail()

,label,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
4306,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4307,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4308,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4309,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4310,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
y_train = df_train['label']
X_train = df_train.loc[:, df_train.columns != "label"]

y_test = df_train['label']
X_test = df_train.loc[:, df_train.columns != "label"]

X_train.to_numpy().shape

(3592, 1024)

# AutoML Libraries

### Versions

In [ ]:
import sys
sys.version

In [ ]:
import tpot
tpot.__version__

In [ ]:
from autogluon import tabular
tabular.__version__

In [ ]:
import flaml
flaml.__version__

### Time Limit

In [7]:
time_min = 30
time_sec = time_min * 60


## TPOT

In [8]:
from tpot import TPOTClassifier

In [9]:
%%time
pipeline_optimizer = TPOTClassifier(verbosity=3, max_time_mins=time_min, n_jobs=-1)
pipeline_optimizer.fit(X_train, y_train)

32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]

Skipped pipeline #33 due to time out. Continuing to the next pipeline.
Skipped pipeline #50 due to time out. Continuing to the next pipeline.
Skipped pipeline #58 due to time out. Continuing to the next pipeline.
Skipped pipeline #94 due to time out. Continuing to the next pipeline.
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=1 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _m

TPOTClassifier(max_time_mins=30, n_jobs=-1, verbosity=3)

In [10]:
print(pipeline_optimizer.score(X_test, y_test))

/opt/conda/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  warnings.warn(


0.9387527839643652


In [11]:
pipeline_optimizer.export('tpot_exported_pipeline.py')

## AutoGluon

In [12]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [13]:
train_data = TabularDataset(df_train)
test_data = TabularDataset(df_test)

In [14]:
%%time
predictor = TabularPredictor(label='label').fit(train_data=train_data, verbosity=4, time_limit=time_sec, presets='best_quality')
# model will decide how many cores to use

No path specified. Models will be saved in: "AutogluonModels/ag-20230922_090845/"
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'included_model_types': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Saving AutogluonModels/ag-2023

CPU times: user 3min 11s, sys: 1min 39s, total: 4min 51s
Wall time: 26min 20s


In [15]:
predictor.evaluate(test_data, silent=False)

Loading: AutogluonModels/ag-20230922_090845/models/LightGBMXT_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/WeightedEnsemble_L2/model.pkl
Evaluation: accuracy on test data: 0.9401947148817803
Evaluations on test data:
{
    "accuracy": 0.9401947148817803,
    "balanced_accuracy": 0.5731151883196589,
    "mcc": 0.29154867707650123,
    "roc_auc": 0.8268622004005427,
    "f1": 0.2456140350877193,
    "precision": 0.6363636363636364,
    "recall": 0.15217391304347827
}


{'accuracy': 0.9401947148817803,
 'balanced_accuracy': 0.5731151883196589,
 'mcc': 0.29154867707650123,
 'roc_auc': 0.8268622004005427,
 'f1': 0.2456140350877193,
 'precision': 0.6363636363636364,
 'recall': 0.15217391304347827}

In [16]:
predictor.leaderboard(test_data, silent=True)

Loading: AutogluonModels/ag-20230922_090845/models/LightGBMXT_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/LightGBM_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/RandomForestGini_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/RandomForestEntr_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/CatBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/XGBoost_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/LightGBMLarge_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20230922_090845/models/WeightedEnsemble_L2/model.pkl
Model scores:
{'LightGBMX

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L1,0.944367,0.934020,1.554220,1.328699,155.791217,1.554220,1.328699,155.791217,1,True,9
1,ExtraTreesGini_BAG_L1,0.942976,0.932350,0.336041,1.054570,2.312812,0.336041,1.054570,2.312812,1,True,6
2,ExtraTreesEntr_BAG_L1,0.942976,0.932350,0.360988,0.942730,1.975779,0.360988,0.942730,1.975779,1,True,7
3,RandomForestEntr_BAG_L1,0.941586,0.933463,0.307003,0.980565,2.007208,0.307003,0.980565,2.007208,1,True,4
4,RandomForestGini_BAG_L1,0.941586,0.932906,0.309322,0.952581,2.328351,0.309322,0.952581,2.328351,1,True,3
5,CatBoost_BAG_L1,0.941586,0.932906,4.400525,10.742911,180.217100,4.400525,10.742911,180.217100,1,True,5
6,LightGBM_BAG_L1,0.940195,0.935690,1.017484,2.226668,58.388364,1.017484,2.226668,58.388364,1,True,2
7,LightGBMXT_BAG_L1,0.940195,0.935690,1.075064,2.383983,58.157029,1.075064,2.383983,58.157029,1,True,1
8,WeightedEnsemble_L2,0.940195,0.935690,1.080796,2.392670,60.012530,0.005731,0.008687,1.855501,2,True,12
9,LightGBMLarge_BAG_L1,0.938804,0.934855,1.433577,2.203835,117.427994,1.433577,2.203835,117.427994,1,True,11


# FLAML

In [17]:
from flaml import AutoML
from sklearn.metrics import accuracy_score

In [ ]:
%%time
automl = AutoML()
automl.fit(X_train, y_train, task="classification", time_budget=time_sec, verbose=9999, ensemble=True)
# uses all cores by default

[flaml.automl.logger: 09-22 09:35:31] {1679} INFO - task = classification
[flaml.automl.logger: 09-22 09:35:31] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 09-22 09:35:31] {1788} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 09-22 09:35:31] {1900} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 09-22 09:35:31] {2218} INFO - iteration 0, current learner lgbm
[flaml.tune.tune: 09-22 09:35:31] {805} INFO - trial 1 config: {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 20, 'learning_rate': 0.09999999999999995, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0}
[flaml.tune.tune: 09-22 09:35:32] {197} INFO - result: {'pred_time': 1.3683875203841545e-05, 'wall_clock_time': 5.874639987945557, 'metric_for_logging': {'pred_time': 1.3683875203841545e-05}, 'val_loss': 0.3371834396384621, 'trained_estimator': <flaml.automl.mod

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.tune.tune: 09-22 09:35:34] {197} INFO - result: {'pred_time': 4.591096324610966e-05, 'wall_clock_time': 8.379932880401611, 'metric_for_logging': {'pred_time': 4.591096324610966e-05}, 'val_loss': 0.41041591556958873, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fb613387640>, 'training_iteration': 0, 'config': {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 0.9999999999999993, 'learning_rate': 0.09999999999999995, 'subsample': 1.0, 'colsample_bylevel': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0}, 'config/n_estimators': 4, 'config/max_leaves': 4, 'config/min_child_weight': 0.9999999999999993, 'config/learning_rate': 0.09999999999999995, 'config/subsample': 1.0, 'config/colsample_bylevel': 1.0, 'config/colsample_bytree': 1.0, 'config/reg_alpha': 0.0009765625, 'config/reg_lambda': 1.0, 'experiment_tag': 'exp', 'time_total_s': 1.501417636871338}
[flaml.tune.tune: 09-22 09:35:34] {197} INFO - result: {'pred_time'

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.tune.tune: 09-22 09:35:38] {197} INFO - result: {'pred_time': 4.6257594086550165e-05, 'wall_clock_time': 12.554547548294067, 'metric_for_logging': {'pred_time': 4.6257594086550165e-05}, 'val_loss': 0.4495247741687022, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fb613387dc0>, 'training_iteration': 0, 'config': {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 3.815612027960909, 'learning_rate': 0.03859136192132085, 'subsample': 1.0, 'colsample_bylevel': 0.8148474110627004, 'colsample_bytree': 0.9777234800442423, 'reg_alpha': 0.0009765625, 'reg_lambda': 5.525802807180917}, 'config/n_estimators': 4, 'config/max_leaves': 4, 'config/min_child_weight': 3.815612027960909, 'config/learning_rate': 0.03859136192132085, 'config/subsample': 1.0, 'config/colsample_bylevel': 0.8148474110627004, 'config/colsample_bytree': 0.9777234800442423, 'config/reg_alpha': 0.0009765625, 'config/reg_lambda': 5.525802807180917, 'experiment_tag': 'exp', 'time_total_s'

...

[flaml.tune.tune: 09-22 09:39:50] {197} INFO - result: {'pred_time': 6.09387277988305e-05, 'wall_clock_time': 264.0940535068512, 'metric_for_logging': {'pred_time': 6.09387277988305e-05}, 'val_loss': 0.18431557783131994, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fb612793520>, 'training_iteration': 0, 'config': {'n_estimators': 139, 'max_leaves': 136, 'min_child_weight': 0.27084276373818733, 'learning_rate': 0.0484091845623208, 'subsample': 0.827351358517848, 'colsample_bylevel': 0.35468713776971533, 'colsample_bytree': 0.6256302030953867, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.3630887969527579}, 'config/n_estimators': 139, 'config/max_leaves': 136, 'config/min_child_weight': 0.27084276373818733, 'config/learning_rate': 0.0484091845623208, 'config/subsample': 0.827351358517848, 'config/colsample_bylevel': 0.35468713776971533, 'config/colsample_bytree': 0.6256302030953867, 'config/reg_alpha': 0.0009765625, 'config/reg_lambda': 1.3630887969527579, '

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.tune.tune: 09-22 09:40:00] {197} INFO - result: {'pred_time': 5.547810983379073e-05, 'wall_clock_time': 274.3884439468384, 'metric_for_logging': {'pred_time': 5.547810983379073e-05}, 'val_loss': 0.2386212818746552, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fb612685ee0>, 'training_iteration': 0, 'config': {'n_estimators': 62, 'max_leaves': 309, 'min_child_weight': 0.08786481946462457, 'learning_rate': 0.02398777014900063, 'subsample': 0.8857073593591062, 'colsample_bylevel': 0.4491830868977382, 'colsample_bytree': 0.7118212477763237, 'reg_alpha': 0.0009765625, 'reg_lambda': 5.163353258094623}, 'config/n_estimators': 62, 'config/max_leaves': 309, 'config/min_child_weight': 0.08786481946462457, 'config/learning_rate': 0.02398777014900063, 'config/subsample': 0.8857073593591062, 'config/colsample_bylevel': 0.4491830868977382, 'config/colsample_bytree': 0.7118212477763237, 'config/reg_alpha': 0.0009765625, 'config/reg_lambda': 5.163353258094623, 'e

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[flaml.tune.tune: 09-22 09:40:56] {197} INFO - result: {'pred_time': 7.468240037115658e-05, 'wall_clock_time': 330.3612959384918, 'metric_for_logging': {'pred_time': 7.468240037115658e-05}, 'val_loss': 0.2060513662087875, 'trained_estimator': <flaml.automl.model.XGBoostSklearnEstimator object at 0x7fb6126851c0>, 'training_iteration': 0, 'config': {'n_estimators': 311, 'max_leaves': 60, 'min_child_weight': 0.8348711477051795, 'learning_rate': 0.09769349695417481, 'subsample': 0.7689953576765899, 'colsample_bylevel': 0.26019118864169244, 'colsample_bytree': 0.5394391584144498, 'reg_alpha': 0.0044372497585360125, 'reg_lambda': 0.35984581637239343}, 'config/n_estimators': 311, 'config/max_leaves': 60, 'config/min_child_weight': 0.8348711477051795, 'config/learning_rate': 0.09769349695417481, 'config/subsample': 0.7689953576765899, 'config/colsample_bylevel': 0.26019118864169244, 'config/colsample_bytree': 0.5394391584144498, 'config/reg_alpha': 0.0044372497585360125, 'config/reg_lambda': 0

In [ ]:
y_pred = automl.predict(X_test)
accuracy_score(y_test,y_pred) 

In [ ]:
print(automl.model.estimator)

In [ ]:
print(automl.best_estimator)
print(automl.best_config)